###### [NOTE] The original input increments by 0.5, but for numerical stability I multiple all by 2, and round to the nearest integer. so now instead of 50*50*50, it is 100*100*100

In [ ]:
import sys, os, glob
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

pd.set_option('display.max_colwidth', -1)

In [ ]:
# you can later use this array to iterate through all files
files = [file for file in glob.glob("result/*.txt")]
print ("NUMBER OF FILES IN RESULT FOLDER: ", len(files))

###### FUNCTIONS

In [ ]:
def getBuildingHeights(filename):
    '''
    input: name of result file
    output: array of 3 heights of our target building (i.e [3,27,6])
    '''
    return filename.split("/")[-1].replace(".txt", "").split(",")

def createColumnMapper():
    mapper = {0: "x", 1:"y", 2:"z"}
    month = [calendar.month_name[i+1] for i in range(12)]
    for i in range(3, 3+12):
        mapper[i] = month[i-3]
    return mapper

def convertToDF(fname):
    '''
    input: name of result file
    output: pandas df
    '''
    with open(fname) as f:
        content = f.readlines()   
    content = np.array([line.replace("\n", "").split(",") for line in content[1:]])
    return pd.DataFrame(data=content)

def matplotlibViz(df, angle1=None, angle2=None, s=1, z=None):
    '''
    Visualizes in 3D a dataframe
    input: dataframe
    output: None
    '''
    color = {0: 'blue', 1:'red'}
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='3d')
    df = df.loc[(df['building'] == 1) & (df['z'] == z)] if z != None else df.loc[df['building'] == 1]
    df['moved'] = df['moved'].apply(lambda x: color[int(x)])
    ax.scatter(df['x'], df['y'], df['z'], s=s, color=df['moved'])
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.view_init(angle1, angle2)
    plt.title("3-D rendered building")
    plt.show()
    
def convertToGridWithoutPad(df, size=(100,100,100), increment=1):
    '''
    zero pad all the coordinates that do not have radiation values (even the coordinates inside the building are zero padded).
    input: dataframe
    output: dataframe
    
    '''
    res = []
    temp = df_building[['x', 'y', 'z']]*2
    # NEW RESULT WITH RAD VALUE (CHANGES START HERE)
    temp['rad'] = df_building[['Annual Sum']]
    temp = temp.apply(pd.to_numeric, errors='coerce', downcast="integer")
    # CREATE HASHMAP FOR RADIATION
    hashmap = {}
    for index, row in temp.iterrows():
        hashmap[tuple(row[:-1])] = row[-1]
    
    for z in range(0, size[0]+increment, increment):
        for y in range(0, size[0]+increment, increment):
            for x in range(0, size[0]+increment, increment):
                if (x,y,z) in hashmap:
                    res.append([x,y,z,hashmap[(x,y,z)], 1, 1]) # FOURTH COLUMN WE ADD RADIATION
                else:
                    res.append([x,y,z,0,0, 1]) # FOURTH COLUMN WE ADD RADIATION, BUT ITS ALL ZERO BECAUSE IT'S INSIDE THE BUILDING
    
    return pd.DataFrame(data=res, columns=['x', 'y', 'z', 'rad', 'building', 'moved'])            
                    
    
def convertToGrid(df, size=(100,100,100), increment=1):
    '''
    zero pad all the coordinates that do not have radiation values (even the coordinates inside the building are zero padded).
    input: dataframe
    output: dataframe
    
    '''
    res = []
    marked = set()
    
    temp = df_building[['x', 'y', 'z']]*2
    temp['rad'] = df_building[['Annual Sum']]
    temp = temp.apply(pd.to_numeric, errors='coerce', downcast="integer")
    # CREATE HASHMAP FOR RADIATION
    hashmap = {}
    for index, row in temp.iterrows():
        hashmap[tuple(row[:-1])] = row[-1]
    
    temp = False
    for z in range(0, size[0]+increment, increment):
        for y in range(0, size[0]+increment, increment):
            for x in range(0, size[0]+increment, increment):
                if (x,y,z) in hashmap:
                    if temp:
                        res.append([x,y,z,hashmap[(x,y,z)], temp])
                        temp = not temp
                    else:
                        temp = not temp
                        res.append([x,y,z,hashmap[(x,y,z)], temp])
                else:
                    res.append([x,y,z,0,temp]) 
    
    return pd.DataFrame(data=res, columns=['x', 'y', 'z', 'rad', 'building'])

###### CONVERTED DATAFRAME

In [ ]:
FILE_NAME = "result/6,18,12.txt"
df = convertToDF(FILE_NAME)
df_building = df.rename(index=int, columns=createColumnMapper()).rename(index=int, columns={15:"Annual Sum"}).apply(pd.to_numeric, errors='coerce')
df_building.head(5)

###### PADDING

In [ ]:
def matplotlibViz(df, angle1=None, angle2=None, s=1, z=None):
    '''
    Visualizes in 3D a dataframe
    input: dataframe
    output: None
    '''
    color = {0: 'blue', 1:'red', 2:'green', 3: 'black'}
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='3d')
    df = df.loc[(df['building'] == 1) & (df['z'] == z)] if z != None else df.loc[df['building'] == 1]
    df['moved'] = df['moved'].apply(lambda x: color[int(x)])
    ax.scatter(df['x'], df['y'], df['z'], s=s, color=df['moved'])
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.view_init(angle1, angle2)
    plt.title("3-D rendered building")
    plt.show()

def pushSurfacesInside(df, size=(100,100,100), increment=1):
    res = []
    temp = df_building[['x', 'y', 'z']]*2
    temp['rad'] = df_building[['Annual Sum']]
    temp = temp.apply(pd.to_numeric, errors='coerce', downcast="integer")
    # CREATE HASHMAP FOR RADIATION
    radiationMap = {}
    for index, row in temp.iterrows():
        radiationMap[tuple(row[:-1])] = row[-1]
    
    # X - Axis PUSH INSIDE
    for z in range(0, size[0]+increment, increment):
        for y in range(0, size[0]+increment, increment):
            for x in range(0, size[0]+increment, increment):
                if (x,y,z) in radiationMap and (x+increment*2, y, z) not in radiationMap:
                    res.append([x+1,y,z, radiationMap[(x,y,z)], 1, 1])
                    break
                elif (x,y,z) in radiationMap and (x+increment*2, y, z) in radiationMap:
                    break
            for x in reversed(range(0, size[0]+increment, increment)):
                if (x,y,z) in radiationMap and (x-increment*2, y, z) not in radiationMap:
                    res.append([x-1,y,z, radiationMap[(x,y,z)], 1, 2])
                    break
                elif (x,y,z) in radiationMap and (x-increment*2, y, z) in radiationMap:
                    break
    
    # Y - Axis PUSH INSIDE
    for z in range(0, size[0]+increment, increment):
        for x in range(0, size[0]+increment, increment):
            for y in range(0, size[0]+increment, increment):
                if (x,y,z) in radiationMap and (x, y+increment*2, z) not in radiationMap:
                    res.append([x,y+1,z, radiationMap[(x,y,z)], 1, 1])
                    break
                elif (x,y,z) in radiationMap and (x, y+increment*2, z) in radiationMap:
                    break
            for y in reversed(range(0, size[0]+increment, increment)):
                if (x,y,z) in radiationMap and (x, y-increment*2, z) not in radiationMap:
                    res.append([x,y-1,z, radiationMap[(x,y,z)], 1, 2])
                    break
                elif (x,y,z) in radiationMap and (x, y-increment*2, z) in radiationMap:
                    break
    
    # Z - Axis PUSH INSIDE
    for x in range(0, size[0]+increment, increment):
        for y in range(0, size[0]+increment, increment):
            for z in range(0, size[0]+increment, increment):
                if (x,y,z) in radiationMap and (x, y, z+increment*2) not in radiationMap:
                    res.append([x,y,z+1, radiationMap[(x,y,z)], 1, 0])
                    break
                elif (x,y,z) in radiationMap and (x, y, z+increment*2) in radiationMap:
                    break
            for z in reversed(range(0, size[0]+increment, increment)):
                if (x,y,z) in radiationMap and (x, y, z-increment*2) not in radiationMap:
                    res.append([x,y,z-1, radiationMap[(x,y,z)], 1, 0])
                    break
                elif (x,y,z) in radiationMap and (x, y, z-increment*2) in radiationMap:
                    break
                    
    return pd.DataFrame(data=res, columns=['x', 'y', 'z', 'Annual Sum', 'building', 'moved'])  

In [ ]:
df_building_original = convertToGridWithoutPad(df_building)

In [ ]:
matplotlibViz(df_building_original, s=1, z=0) # only view slice of z coordinate (horizontal slice at z=1)

In [ ]:
df_building_pushed = pushSurfacesInside(df_building)

In [ ]:
matplotlibViz(df_building_pushed, s=3, z=21) # only view slice of z coordinate (horizontal slice at z=1)

In [ ]:
df_building_pushed.head(5)

In [ ]:
import collections

def removeOverlappingPoints(df):
    '''
    REMOVES OVERLAPPING COORDINATES THAT HAVE MORE THAN ONE VALUE
    
    '''
    everyDataPointIncludingOverlaps =  [tuple(row[:3]) for idx, row in df.iterrows()]
    hashmap = {tuple(row[:3]): row[3] for index, row in df.iterrows()}  # CREATE HASHMAP FOR RADIATION
    counter = collections.Counter(everyDataPointIncludingOverlaps)
    uniqueCoordinates = [k for k, v in counter.items() if v == 1]  
    print ('NUMBER OF UNIQUE COORDINATES' , len(uniqueCoordinates))
    res = [list(row) for idx, row in df.iterrows() if tuple(row[:3]) in uniqueCoordinates]   
    return pd.DataFrame(data=res, columns=['x', 'y', 'z', 'Annual Sum', 'building', 'moved']) 


In [ ]:
df_building_pushed_overlap_removed = removeOverlappingPoints(df_building_pushed)

In [ ]:
print (df_building_pushed_overlap_removed.shape)

In [ ]:
matplotlibViz(df_building_pushed_overlap_removed, s=3, z=21) # only view slice of z coordinate (horizontal slice at z=1)

In [ ]:
def convertToGrid(df, size=(100,100,100), increment=1):
    '''
    zero pad all the coordinates that do not have radiation values (even the coordinates inside the building are zero padded).
    input: dataframe
    output: dataframe
    
    '''
    res = []
    # CREATE HASHMAP FOR RADIATION
    hashmap = {}
    for index, row in df.iterrows():
        hashmap[tuple(row[:3])] = row[4:]
    
    for z in range(0, size[0]+increment, increment):
        for y in range(0, size[0]+increment, increment):
            padding = False
            for x in range(0, size[0]+increment, increment):
                if (x,y,z) in hashmap and (x+increment*2, y, z) not in hashmap:
                    x += increment*2
                    while ((x,y,z) not in hashmap):
                        res.append([x,y,z, 0, 1, 3])
                        x += increment*2
                    break
                elif (x,y,z) in hashmap and (x+increment*2, y, z) in hashmap:
                    break
    original = [list(row) for index, row in df.iterrows()]
    return pd.DataFrame(data=res+original, columns=['x', 'y', 'z', 'rad', 'building', 'moved'])

In [ ]:
[1,2] + [3,4]

In [ ]:
final = convertToGrid(df_building_pushed_overlap_removed)

In [ ]:
matplotlibViz(final, s=3, z=21) # only view slice of z coordinate (horizontal slice at z=1)